In [ ]:
import re
import morfessor

# to create training data for morfessor

def createTrainingData(input):
    words = []

    with open(input, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("\\t"):
                tokens = line[2:].strip().split()
                words.extend(tokens)
    return [(1.0, word) for word in words]


In [ ]:
trainingData = createTrainingData("ntu-train-track1-uncovered")  # change the input file there

model = morfessor.BaselineModel()
model.load_data(trainingData)
model.train_batch()

In [ ]:
output_lines = []

# segment each word 

with open("ntu-train-track1-uncovered", "r", encoding="utf-8") as f:
    for line in f:
        output_lines.append(line.rstrip('\n'))
        if line.startswith("\\t"):
            sentence = line[2:].strip()
            segmented = []
            for token in sentence.split():
                if len(token) == 1 or token == "...":
                    segmented.append(token)
                    continue

                clean = re.sub("[,\".]", "", token)
                morphs = model.viterbi_segment(clean)[0]
                joined = "-".join(morphs)

                if joined.endswith("-"):
                    joined = joined[:-1]

                if joined.startswith("'"):

                    joined = "'" + joined[1:].lower()
                else:
                    joined = joined.lower()

                joined = joined.replace("--", "-")

                segmented.append(joined)

            output_lines.append("\\m " + " ".join(segmented))

#store output in a file
with open("output", "w", encoding="utf-8") as f:
    for line in output_lines:
        f.write(line + "\n")